## Imports and Dataset


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

# import lightgbm as lgb
from IPython.display import Markdown, display

import warnings

warnings.filterwarnings("ignore")

import utils.ml_utils as mlu


In [ ]:
df = pd.read_csv("../data/clean/df-train.csv")
amount_outliers_df = pd.read_csv("../data/clean/df-amount_outliers.csv")
age_loan_outliers_df = pd.read_csv("../data/clean/df-adult_loans.csv")


## ML Models


### Apply Function


In [ ]:
def apply(
    df,
    model_instance,
    parameter_grid,
    cross_validation=StratifiedKFold(n_splits=5),
    feature_selection=False,
    filter=True,
    oversample=False,
    scaler=True,
):
    return mlu.apply_cv(
        df,
        model_instance,
        parameter_grid,
        cross_validation,
        feature_selection,
        filter,
        oversample,
        scaler,
    )


##### Decision Tree


In [ ]:
"""
Description:
Non-parametric supervised learning method (used for classification).
The goal is to create a model that predicts a variable (status) by learning simple decision rules inferred from the data features.
A tree can be seen as a piecewise constant approximation.

Interesting parameters to tune:
    - criterion (function used to measure the quality of a split): gini (default), entropy or log_loss
    - splitter (strategy used to choose a split): best (default) or random
    - random_state (controls the randomness of the estimator)
"""


def apply_decision_tree(df):
    parameter_grid = {
        "criterion": ["gini", "entropy", "log_loss"],
        "splitter": ["best", "random"],
        "max_depth": range(1, 7),
    }
    return (
        apply(
            df,
            tree.DecisionTreeClassifier,
            parameter_grid,
            oversample=oversample,
            feature_selection=feature_selection,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


##### Logistic Regression


In [ ]:
"""
Description:
Logistic regression, despite its name, is a linear model for classification rather than regression.
Logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier.
In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function.

Interesting parameters to tune:
    - max_iter (maximum number of iterations taken for the solvers to converge, default 100)
    - solver (algorithm to use in the optimization problem): lbfgs (default)

Some notes on the solver:
    - for small datasets, liblinear is a good choice, whereas sag and saga are faster for large ones;
    - for multiclass problems, only newton-cg, sag, saga and lbfgs handle multinomial loss;
    - liblinear is limited to one-versus-rest schemes.
"""


def apply_logistic_regression(df):
    parameter_grid = {
        "penalty": ["l2", "none"],
        "C": [0.01, 0.05, 0.1, 0.2, 0.5, 1.0],
        "solver": ["newton-cg", "lbfgs", "sag", "saga"],
        "class_weight": ["balanced", None],
        "max_iter": [500, 1000, 5000, 10000],
    }
    return (
        apply(
            df,
            LogisticRegression,
            parameter_grid,
            oversample=oversample,
            feature_selection=feature_selection,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


##### Naive Bayes


In [ ]:
"""
Description:
Naive Bayes methods are a set of supervised learning algorithms based on applying Bayes’ 
theorem with the "naive" assumption of conditional independence between every pair of features given the value of the class variable.

Draw the Gaussian Distribution to each of the independent variables and apply the likelihood of each given value to the prior probability.
"""


def apply_naive_bayes(df):
    return (
        apply(
            df,
            GaussianNB,
            {},
            oversample=oversample,
            feature_selection=feature_selection,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


##### K-Nearest Neighbors


In [ ]:
"""
Description:
Neighbors-based classification is a type of instance-based learning or non-generalizing learning:
it does not attempt to construct a general internal model, but simply stores instances of the training data.

Classification is computed from a simple majority vote of the nearest neighbors of each point:
a query point is assigned the data class which has the most representatives within the nearest neighbors of the point.
"""


def apply_k_nearest_neighbours(df):
    parameter_grid = {
        "n_neighbors": [4, 5, 6, 7, 10, 15],
        "leaf_size": [5, 10, 15, 20, 50, 100],
        "n_jobs": [-1],
        "algorithm": ["auto"],
    }
    return (
        apply(
            df,
            KNeighborsClassifier,
            parameter_grid,
            oversample=oversample,
            feature_selection=feature_selection,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


##### Random Forest


In [ ]:
"""
Description:
A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset
 and uses averaging (votes) to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree.

Interesting parameters to tune:
    - n_estimators (number of trees in the forest, default 100)
    - criterion (function used to measure the quality of a split): gini (default), entropy or log_loss
    - max_depth (maximum depth of the tree, default None)
    - n_jobs
"""


def apply_random_forest(df):

    parameter_grid = {
        "n_estimators": [10, 50, 100, 200],
        "max_depth": [5, 10, None],
        "n_jobs": [-1],  # Use all cores
        "criterion": ["gini", "entropy"],
    }
    return (
        apply(
            df,
            RandomForestClassifier,
            parameter_grid,
            oversample=oversample,
            feature_selection=feature_selection,
            scaler=False,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


##### Gradient Boosting


In [ ]:
"""
Description:
Gradient Tree Boosting or Gradient Boosted Decision Trees (GBDT) is a generalization of boosting to arbitrary differentiable loss functions.
GBDT is an accurate and effective off-the-shelf procedure that can be used for both regression and classification problems
 in a variety of areas including Web search ranking and ecology.
Similar to Random forest with the improvement from tree to tree (loss decrease)
"""


def apply_gradient_boosting(df):
    parameter_grid = {
        "n_estimators": [int(x) for x in range(2, 14, 2)],
        "learning_rate": [0.1, 0.3, 0.5, 0.7],
        "loss": ["deviance", "exponential", "log_loss"],
        "criterion": ["friedman_mse", "squared_error", "mse"],
        "min_samples_split": [4, 6, 8],
        "min_samples_leaf": [2, 4, 6],
    }
    return (
        apply(
            df,
            GradientBoostingClassifier,
            parameter_grid,
            oversample=oversample,
            feature_selection=feature_selection,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


##### SVC


In [ ]:
"""
Description:
SVC separates the set of points (items) with a line or plane into different areas of the space.
These areas allow us to predict if a given point is of a specific class or not.
"""


def apply_svc(df):
    parameter_grid = {
        "C": [1, 10, 50],
        "gamma": [0.001, 0.0001, 0.01, 1, "scale", "auto"],
        "kernel": ["linear", "poly", "rbf", "sigmoid"],
        "degree": [1, 2, 3, 4, 5, 6, 7],
        "coef0": [0.0, 0.1, 0.3, 0.5, 0.7],
        "max_iter": [1, 2, 3, 5, 7, 10],
        "decision_function_shape": ["ovo", "ovr"],
        "class_weight": [None, "balanced", dict],
    }
    return (
        apply(
            df,
            SVC,
            parameter_grid,
            oversample=oversample,
            feature_selection=feature_selection,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


##### MLP


In [ ]:
"""
Description:
"""


def apply_mlp(df):
    parameter_grid = {
        "hidden_layer_sizes": [(3, 5, 8, 13, 21, 34)],
        "activation": ["identity", "logistic", "tanh", "relu"],
        "solver": ["lbfgs", "sgd", "adam"],
        "learning_rate": ["constant", "invscaling", "adaptive"],
        "nesterovs_momentum": [True, False],
        "early_stopping": [False, True],
    }
    return (
        apply(
            df,
            MLPClassifier,
            parameter_grid,
            oversample=oversample,
            feature_selection=feature_selection,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


##### XGBOOST


In [ ]:
def apply_xgboost(df):
    parameter_grid = {
        "min_child_weight": range(1, 6, 2),
        "gamma": [0, 0.2, 0.5, 1, 1.5, 3],
        "max_depth": [5, 8, 10, 15],
        "reg_alpha": [1e-5, 1e-2, 0.1, 1],
    }
    return (
        apply(
            df,
            XGBClassifier,
            parameter_grid,
            oversample=oversample,
            feature_selection=feature_selection,
        )
        for oversample, feature_selection in (
            (False, False),
            (False, True),
            (True, False),
            (True, True),
        )
    )


### Comparing Models


In [ ]:
import seaborn as sb


def compare_models(data):
    scores = {
        "Decision Tree": data["dt"],
        "Logistic Regression": data["lr"],
        "Naive Bayes": data["nb"],
        "K-nearest Neighbours": data["knn"],
        "Random Forest": data["rf"],
        "xgboost": data["xg"],
        "Gradient Boosting": data["gb"],
        "SVC": data["svc"],
    }

    x_axis_labels = [
        "No Feature selection/No oversampling",
        "Feature Selection",
        "Oversampling",
        "Feature Selection/Oversampling",
    ]
    y_axis_labels = scores.keys()

    plt.figure(figsize=(10, 8))

    scores_array = np.array([[score for score in model] for model in scores.values()])

    sb.set(font_scale=1.3)
    sb.heatmap(
        scores_array,
        annot=True,
        linewidths=0.5,
        vmax=1,
        square=True,
        xticklabels=x_axis_labels,
        yticklabels=y_axis_labels,
        cbar=False,
    )
    plt.title("ROC-AUC")

    plt.xticks(rotation=45, horizontalalignment="right")
    plt.show()


In [ ]:
""" dfs = {
  'Main': df, 
  #'Amount outliers removed': amount_outliers_df, 
  #'Age outliers removed': age_loan_outliers_df,
  } 


models_comparison = {}

for key, value in dfs.items():
  display(Markdown(f"# **{key} dataframe** "))

  display(Markdown(f"## Random Forest "))
  rf, rf_fs, rf_os, rf_fs_os = apply_random_forest(value)
  display(Markdown("---"))

  display(Markdown(f"## XGBoost "))
  xg, xg_fs, xg_os, xg_fs_os = apply_xgboost(value)
  display(Markdown("---"))

  display(Markdown(f"## Decision Tree "))
  dt, dt_fs, dt_os, dt_fs_os = apply_decision_tree(value)
  display(Markdown("---"))

  display(Markdown(f"## Gradient Boosting "))
  gb, gb_fs, gb_os, gb_fs_os = apply_gradient_boosting(value)
  display(Markdown("---"))

  display(Markdown(f"## K Nearest Neighbours "))
  knn, knn_fs, knn_os, knn_fs_os = apply_k_nearest_neighbours(value)
  display(Markdown("---"))

  display(Markdown(f"## Logistic Regression "))
  lr, lr_fs, lr_os, lr_fs_os = apply_logistic_regression(value)
  display(Markdown("---"))

  display(Markdown(f"## Naive Bayes "))
  nb, nb_fs, nb_os, nb_fs_os = apply_naive_bayes(value)
  display(Markdown("---"))

  display(Markdown(f"## SVC "))
  svc, svc_fs, svc_os, svc_fs_os = apply_svc(value)
  display(Markdown("---"))

  display(Markdown(f"## MLP "))
  mlp, mlp_fs, mlp_os, mlp_fs_os = apply_mlp(value)
  display(Markdown("---"))
  

  models_comparison[key] = {
                          'dt':[dt, dt_fs, dt_os, dt_fs_os],
                          'gb':[gb, gb_fs, gb_os, gb_fs_os], 
                          'knn':[knn, knn_fs, knn_os, knn_fs_os], 
                          'lr':[lr, lr_fs, lr_os, lr_fs_os], 
                          'nb':[nb, nb_fs, nb_os, nb_fs_os], 
                          'xg':[xg, xg_fs, xg_os, xg_fs_os], 
                          'svc':[svc, svc_fs, svc_os, svc_fs_os],
                          'mlp':[mlp, mlp_fs, mlp_os, mlp_fs_os],
                          'rf':[rf, rf_fs, rf_os, rf_fs_os]} """


In [ ]:
display(Markdown(f"## Random Forest "))
rf, rf_fs, rf_os, rf_fs_os = apply_random_forest(df)
display(Markdown("---"))

In [ ]:
display(Markdown(f"## XGBoost "))
xg, xg_fs, xg_os, xg_fs_os = apply_xgboost(df)
display(Markdown("---"))

In [ ]:
display(Markdown(f"## Decision Tree "))
dt, dt_fs, dt_os, dt_fs_os = apply_decision_tree(df)
display(Markdown("---"))

In [ ]:
display(Markdown(f"## Gradient Boosting "))
gb, gb_fs, gb_os, gb_fs_os = apply_gradient_boosting(df)
display(Markdown("---"))

In [ ]:
display(Markdown(f"## K Nearest Neighbours "))
knn, knn_fs, knn_os, knn_fs_os = apply_k_nearest_neighbours(df)
display(Markdown("---"))

In [ ]:
display(Markdown(f"## Logistic Regression "))
lr, lr_fs, lr_os, lr_fs_os = apply_logistic_regression(df)
display(Markdown("---"))

In [ ]:
display(Markdown(f"## Naive Bayes "))
nb, nb_fs, nb_os, nb_fs_os = apply_naive_bayes(df)
display(Markdown("---"))

In [ ]:
display(Markdown(f"## SVC "))
svc, svc_fs, svc_os, svc_fs_os = apply_svc(df)
display(Markdown("---"))

In [ ]:
display(Markdown(f"## MLP "))
mlp, mlp_fs, mlp_os, mlp_fs_os = apply_mlp(df)
display(Markdown("---"))

In [ ]:
models_comparison = {}
models_comparison["main"] = {
    "dt": [dt, dt_fs, dt_os, dt_fs_os],
    "gb": [gb, gb_fs, gb_os, gb_fs_os],
    "knn": [knn, knn_fs, knn_os, knn_fs_os],
    "lr": [lr, lr_fs, lr_os, lr_fs_os],
    "nb": [nb, nb_fs, nb_os, nb_fs_os],
    "xg": [xg, xg_fs, xg_os, xg_fs_os],
    #'svc':[svc, svc_fs, svc_os, svc_fs_os],
    "mlp": [mlp, mlp_fs, mlp_os, mlp_fs_os],
    "rf": [rf, rf_fs, rf_os, rf_fs_os],
}


In [ ]:
""" display(Markdown(f"# **Compare models** "))

for key in dfs.keys():
  display(Markdown(f"## {key} "))

  compare_models(models_comparison[key]) """


In [ ]:
compare_models(models_comparison["main"])
